In [246]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import re

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "オータ志木駅前店": "",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [247]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 検索期間: {start_date} ～ {end_date}", f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}")   

    return df

In [248]:
# ブドウシミュレーター
def grape_calc_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.04228,
        "cherryOff": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.040475,
        "cherryOff": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOn": 0.0372,
        "cherryOff": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.04324,
        "cherryOff": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df["Grape_rate"] = df.apply(
        lambda row: calc_grape_rate(row, GRAPE_CONSTANTS, cherry=False), axis=1
    )
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    for i, model in enumerate(model_list):
        print(f"{i+1}: {model}", end=", ")

    return df_pre, model_list

## ピボットテーブル作成


In [ ]:
def create_pivot_table(df, index, columns, reverse=False, margins=True, day_target=None):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]
    
    pivot_results = {}
    cols = ["game", "medals", "BB", "RB"]
    for col in cols:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game, "medals": medals, "medal_rate": medal_rate,
        "bb": bb, "rb": rb, "rb_rate": rb_rate, "total_rate": total_rate,
    }

    return merged, details


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+3), cols=str(cols+3))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## 出力データ

- 期間指定
  - 機種別の分析
  - 島別の分析
  - 台番号分析
  - 日付分析
  - 月分析
  - 過去 n 日の差枚比較
  - 全台系分析


In [ ]:
# hall_name = "コンサートホールエフ成増"
# hall_name = "第一プラザみずほ台店"
# hall_name = "第一プラザ坂戸1000"
# hall_name = "オータ志木駅前店"

hall_name = "EXA FIRST"
model_name = "ジャグラー"


today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 30248 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-21 📅 抽出期間: 2024-11-01 ～ 2025-05-20
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, 

In [250]:
spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
print(f"スプレッドシート: {spreadsheet.title} を開きました。")

スプレッドシート: EXA FIRST を開きました。


## MODEL_RATE


In [ ]:
sheet_name = "MODEL_RATE"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 30248 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-21 📅 抽出期間: 2024-11-01 ～ 2025-05-20
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, ✅ シート「MODEL_RATE」が既に存在します。
✅ シート 'MODEL_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      1.007  0.988  1.002  0.980  1.002  1.001  0.985  1.002   
ウルトラミラクルジャグラー      1.010  1.010  1.007  0.998  1.008  0.994  1.019  0.983   
ゴーゴージャグラー3         1.007  1.011  0.980  0.995  1.009  1.008  0.999  0.994   
ジャグラーガールズ          1.007  1.000  0.976  0.967  0.996  1.001  0.993  0.985   
ハッピージャグラーVIII      0.993  0.959  0.933  1.020  0.997  1.003  1.029  1.056   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.981  0.999  ...  1.001  0.972  1.002  1.025  0.976  0.991   
ウルトラミラクルジャグラー  0.986  0.987  ...  0.993  0.987  1.010  1.037  0.999  0.975   
ゴーゴージャグラー3     0.998  1.003  ...  0.990  0.992  0.996  1.020  0.990  0.978   
ジャグラーガールズ      0.987  0.998  ...  0.993  1.001  1.005  1.008  0.987  0.983   
ハッピージャグラーVIII  0.985  1.002  ...  0.915  0.930  1.002  1.045  0.927  1.027   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.995  1.000  0.989  0.998  
ウルトラミラクルジャグラー  0.990  0.970  0.969  0.998  
ゴーゴージャグラー3     0.987  0.999  0.989  0.998  
ジャグラーガールズ      0.992  0.986  0.993  0.995  
ハッピージャグラーVIII  0.947  0.901    NaN  0.990  

[5 rows x 32 columns]

## ISLAND_RATE


In [ ]:
sheet_name = f"ISLAND_RATE"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.tail(3)

✅ シート「ISLAND_RATE」が既に存在します。
✅ シート 'ISLAND_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
area                                                                           
t: ジャグラーガールズ          1.008  1.004  0.943  0.984  1.008  1.000  0.971  0.994   
u: アイムジャグラーEX-TP      1.008  0.983  1.027  0.972  0.996  0.999  0.975  1.011   
total                 1.016  0.998  0.994  0.996  1.007  1.001  0.999  0.998   

                                ...                                            \
day                   9     10  ...     23     24     25     26     27     28   
area                            ...                                             
t: ジャグラーガールズ      0.989  1.002  ...  1.007  0.979  1.008  0.990  0.992  0.988   
u: アイムジャグラーEX-TP  0.996  0.964  ...  1.000  0.979  1.013  1.004  0.963  0.993   
total             0.991  0.995  ...  0.999  0.987  1.007  1.015  0.998  0.987   

                                              
day                  29     30     31  total  
area                                          
t: ジャグラーガールズ      1.001  0.988  1.006  0.996  
u: アイムジャグラーEX-TP  1.007  0.983  0.984  0.997  
total             0.991  0.994  0.998  1.000  

[3 rows x 32 columns]

## UNIT_RATE


In [ ]:
sheet_name = f"UNIT_RATE"
index = ["area", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.tail()

✅ シート「UNIT_RATE」が既に存在します。
✅ シート 'UNIT_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                            \
day                            1      2      3      4      5      6      7   
a: ゴーゴージャグラー3    1001      1.048  0.976  1.004  1.013  0.970  0.983  1.006   
                 1002      1.048  1.008  1.012  1.024  0.969  1.030  0.973   
                 1003      1.061  0.968  1.006  1.006  1.047  0.967  0.982   
                 1004      1.018  1.019  1.038  0.976  0.980  0.981  1.013   
                 1005      1.037  0.982  1.017  1.054  0.999  0.978  1.008   
...                          ...    ...    ...    ...    ...    ...    ...   
u: アイムジャグラーEX-TP 1153      0.985  0.987  1.071  0.949  0.996  0.983  1.026   
                 1154      1.020  0.987  1.002  0.914  1.000  1.034  0.959   
                 1155      1.008  0.973  0.999  0.999  1.024  0.997  0.966   
total                      1.016  0.998  0.994  0.996  1.007  1.001  0.999   
count_1.05+               23.000  7.000  5.000  6.000  9.000  9.000  6.000   

                                            ...                         \
day                        8      9     10  ...     24      25      26   
a: ゴーゴージャグラー3    1001  0.941  0.978  0.915  ...  0.975   0.995   0.993   
                 1002  0.981  0.994  0.995  ...  1.047   1.015   0.998   
                 1003  0.987  0.920  0.986  ...  1.000   0.948   1.037   
                 1004  1.004  0.990  0.984  ...  0.987   0.984   1.025   
                 1005  0.979  1.008  1.022  ...  0.960   0.925   1.017   
...                      ...    ...    ...  ...    ...     ...     ...   
u: アイムジャグラーEX-TP 1153  0.929  1.016  0.971  ...  0.973   1.025   0.994   
                 1154  1.028  0.916  0.912  ...  0.989   1.018   1.010   
                 1155  0.991  1.006  0.957  ...  0.991   1.033   0.993   
total                  0.998  0.991  0.995  ...  0.987   1.007   1.015   
count_1.05+            9.000  7.000  5.000  ...  6.000  15.000  23.000   

                                                                          
day                        27     28     29      30      31  total 1.05+  
a: ゴーゴージャグラー3    1001   0.959  0.948  0.987   1.005   0.963  0.986     0  
                 1002   0.990  0.967  0.911   0.978   1.027  1.005     0  
                 1003   1.023  0.984  1.009   0.963   0.934  1.012     4  
                 1004   1.093  0.732  1.002   0.983   1.007  1.005     1  
                 1005   0.988  1.049  0.941   0.992   1.062  1.003     3  
...                       ...    ...    ...     ...     ...    ...   ...  
u: アイムジャグラーEX-TP 1153   0.972  1.021  1.037   0.981   0.972  1.001     2  
                 1154   0.959  1.015  0.960   0.931   0.969  0.998     1  
                 1155   0.876  0.973  1.078   1.014   1.022  0.997     1  
total                   0.998  0.987  0.991   0.994   0.998  1.000     0  
count_1.05+            14.000  5.000  8.000  14.000  18.000  0.000    86  

[154 rows x 33 columns]

# 日付、台番号指定

- SHEET_NAME : DAY1，DAY2，… DAY31


In [242]:
index = ["area", "unit_no"]
columns = ["date"]

# for day_target in range(today.day - 1, today.day + 1):
for day_target in [2]:
    merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True, day_target=day_target)
    add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)

merged.head()

✅ シート「DAY2」が既に存在します。
✅ シート 'DAY2' に DataFrame を書き込みました！


GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date                   total  total total total      total   total      total   
area          unit_no                                                           
a: ゴーゴージャグラー3 1001     33660  -2394   121    93      0.976   361.9      157.3   
              1002     18471    428    72    59      1.008   313.1      141.0   
              1003     25252  -2422    89    60      0.968   420.9      169.5   
              1004     30988   1759   131    86      1.019   360.3      142.8   
              1005     21611  -1150    77    62      0.982   348.6      155.5   

                                     GAME              MEDALS  \
date                  2025-05-02 00:00:00 2025-05-02 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   5317                 -68   
              1002                    635                -238   
              1003                   1386                -385   
              1004                   3288                 332   
              1005                    894                -556   

                                       BB  ...          MEDAL_RATE  \
date                  2025-05-02 00:00:00  ... 2024-12-02 00:00:00   
area          unit_no                      ...                       
a: ゴーゴージャグラー3 1001                     21  ...               0.945   
              1002                      2  ...               0.962   
              1003                      4  ...               1.087   
              1004                     13  ...               1.129   
              1005                      2  ...               0.932   

                                  RB_RATE          TOTAL_RATE  \
date                  2024-12-02 00:00:00 2024-12-02 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  356.3               171.6   
              1002                  408.0               174.9   
              1003                  560.2               140.0   
              1004                  370.3               123.4   
              1005                  451.4               173.6   

                                     GAME              MEDALS  \
date                  2024-11-02 00:00:00 2024-11-02 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                   3848                 344   
              1002                   2648                -944   
              1003                   2762               -1006   
              1004                   4862                 912   
              1005                   1082                -679   

                                       BB                  RB  \
date                  2024-11-02 00:00:00 2024-11-02 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                     16                   9   
              1002                      9                   2   
              1003                      8                   4   
              1004                     26                   7   
              1005                      2                   2   

                               MEDAL_RATE             RB_RATE  \
date                  2024-11-02 00:00:00 2024-11-02 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  1.030               427.6   
              1002                  0.881              1324.0   
              1003                  0.879               690.5   
              1004                  1.063               694.6   
              1005                  0.791               541.0   

                               TOTAL_RATE  
date                  2024-11-02 00:00:00  
area          unit_no                      
a: ゴーゴージャグラー3 1001                  153.9  
              1002               

## 履歴の表示